<a href="https://colab.research.google.com/github/cardazuluaga/DLCourse/blob/main/Ej9PrediccionTextoipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.api._v2 import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Datos de entrenamiento
text = """En el actual territorio de Colombia, los conquistadores fundaron numerosos centros urbanos desde los cuales ejercían su poder, delimitados por un complejo sistema de circunscripciones de lugares, parroquias, villas y ciudades."""

# Preprocesamiento del texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Crear secuencias de entrada y salida
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad secuencias para que tengan la misma longitud
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Crear variables de entrada y salida
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Convertir etiquetas a one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Definir el modelo
model = Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_length - 1),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(total_words,activation='softmax')
])

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X, y, epochs=200, verbose=0)

# Función para predecir la siguiente palabra
def predict_next_word(seed_text, model, tokenizer):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted)
    predicted_word = [word for word, index in tokenizer.word_index.items() if index == predicted_word_index]
    return predicted_word[0]


In [ ]:
# Ejemplo de predicción
seed_text = "En el actual territorio de Colombia, los conquistadores fundaron numerosos centros"
next_word = predict_next_word(seed_text, model, tokenizer)
print(f"Siguiente palabra predicha: {next_word}")

Siguiente palabra predicha: urbanos
